In [1]:
import pandas as pd

In [4]:
data = pd.read_csv("a_b test ecom - ab_test 2.csv", sep=',', encoding='windows-1251')

data.head(10)

,id,time,con_treat,page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
5,936923,20:49.1,control,old_page,0
6,679687,26:46.9,treatment,new_page,1
7,719014,48:29.5,control,old_page,0
8,817355,58:09.0,treatment,new_page,1
9,839785,11:06.6,treatment,new_page,1


In [5]:
# смотрим количество столбцов, тип данных в них и наличие пустых строк
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         294478 non-null  int64 
 1   time       294478 non-null  object
 2   con_treat  294478 non-null  object
 3   page       294478 non-null  object
 4   converted  294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [6]:
# смотрим статистические показатели числовых столбцов
data.describe()

,id,converted
count,294478.000000,294478.000000
mean,787974.124733,0.119659
std,91210.823776,0.324563
min,630000.000000,0.000000
25%,709032.250000,0.000000
50%,787933.500000,0.000000
75%,866911.750000,0.000000
max,945999.000000,1.000000


In [7]:
# считаем кол-во записей в данных
data.con_treat.value_counts()

con_treat
treatment    147276
control      147202
Name: count, dtype: int64

In [8]:
data.page.value_counts()

page
old_page    147239
new_page    147239
Name: count, dtype: int64

In [9]:
# смотрим кол-во уникальных id
data.id.nunique()

290584

In [10]:
# оставляем данные только где контрольная группа получила старуюстраницу и целевая новую страницу
data = data.query('(con_treat == "control" and page == "old_page") or (con_treat == "treatment" and page == "new_page")')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290585 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         290585 non-null  int64 
 1   time       290585 non-null  object
 2   con_treat  290585 non-null  object
 3   page       290585 non-null  object
 4   converted  290585 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


In [11]:
# вычисляем кол-во строк для каждой группы и -1 -  удаляем отрицательные значения
n1 = data.query('con_treat == "control"').shape[0]-1
n2 = data.query('con_treat == "treatment"').shape[0]-1
data.query('con_treat == "control"').info()

<class 'pandas.core.frame.DataFrame'>
Index: 145274 entries, 0 to 294476
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         145274 non-null  int64 
 1   time       145274 non-null  object
 2   con_treat  145274 non-null  object
 3   page       145274 non-null  object
 4   converted  145274 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 6.7+ MB


In [12]:
# считаем кол-во конверсий по каждой группе
k1 = data.query('con_treat == "control"').converted.sum()
k2 = data.query('con_treat == "treatment"').converted.sum()
print(k1,k2)

17489 17264


In [13]:
from statsmodels.stats import proportion
import numpy as np
z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]),
                                                   np.array([n1, n2]))
print('Results are ','z_score =%.3f, pvalue = %.3f'%(z_score, z_pvalue))

Results are  z_score =1.312, pvalue = 0.190


In [14]:
import numpy as np
proportion.proportions_chisquare(np.array([k1, k2]),
                                                   np.array([n1, n2]))
# Стат. значимых различий методом пропорций и хи-квадратом не обнаружено

(1.7203163882100383,
 0.189652323949578,
 (array([[ 17489., 127784.],
         [ 17264., 128046.]]),
  array([[ 17374.28744627, 127898.71255373],
         [ 17378.71255373, 127931.28744627]])))